In [4]:
import argparse
import sys
from salad.single_search import parser
# parser = argparse.ArgumentParser(prog=__name__)
# parser.add_argument("input", nargs="?", type=argparse.FileType('rb'), default=sys.stdin)
# parser.add_argument('output', type=str)
# parser.add_argument("--fakes", required=True)
# parser.add_argument("--images", required=True)
# parser.add_argument("--velocity", nargs=2, type=float, default=[0.1, 0.5])
# parser.add_argument("--angle", nargs=2, type=float, default=[120, 240])
# parser.add_argument("--dx", type=float, default=10)
# parser.add_argument("--vote-threshold", type=int, default=25)
# parser.add_argument("--min-points", type=int, default=15)
# parser.add_argument("--processes", "-J", type=int, default=1)
# parser.add_argument("--gather-threshold", nargs="+", type=float, default=[1, 1, 1])


In [21]:
parser._actions[1]

_StoreAction(option_strings=[], dest='input', nargs='?', const=None, default=<_io.TextIOWrapper name='<stdin>' mode='r' encoding='utf-8'>, type=FileType('rb'), choices=None, required=False, help=None, metavar=None)

In [55]:
def command_factory(parser, time="/bin/time --verbose", prefix="salad"):
    def f(**kwargs):
        cmd = []
        if time:
            cmd.append(time)
        cmd.append(prefix)
        cmd.append(parser.prog.split(f"{prefix}.")[1])
        positional = []
        optional = []
        for action in parser._actions:
            if isinstance(action, argparse._StoreAction):
                if len(action.option_strings) == 0: # positional
                    l = positional
                else: # optional
                    l = optional
                    l.append(action.option_strings[0])

                if action.dest in kwargs:
                    v = kwargs[action.dest]
                else:
                    v = action.default

                if v is not None:
                    if v in [sys.stdin, sys.stdout, sys.stderr]:
                        continue
                    
                    if isinstance(v, list):
                        l.extend(v)
                    else:
                        l.append(v)
                elif action.required:
                    raise Exception(f"must pass {action.dest} since it is a required argument and no default is specified")

        cmd = cmd + positional + optional

        return " ".join(map(str, cmd))
    return f

command_factory(parser)(
    input="catalog.pkl",
    output="single_search", 
    fakes="fakes.pkl", 
    images="images.pkl", 
    velocity=[0.1, 0.5]
)

'/bin/time --verbose salad single_search catalog.pkl single_search --fakes fakes.pkl --images images.pkl --velocity 0.1 0.5 --angle 120 240 --dx 10 --vote-threshold 25 --min-points 15 --processes 1 --gather-threshold 1 1 1'

In [51]:
from parsl import File, bash_app
import parsl

In [56]:
bash_app(
    command_factory(parser)
)(
        input=File("catalog.pkl"),
        output="single_search", 
        fakes="fakes.pkl", 
        images="images.pkl", 
        velocity=[0.1, 0.5]
)

RuntimeError: Must first load config